In [1]:
import pandas as pd


/tmp/ipykernel_1314/3622290940.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
import requests

In [5]:
from dotenv import load_dotenv


os.environ["OPENAI_API_TYPE"] = OPENAI_API_TYPE
os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE
os.environ["AZURE_OPENAI_ENDPOINT"] = OPENAI_API_BASE
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_DEPLOYMENT_NAME"] = OPENAI_DEPLOYMENT_NAME

load_dotenv()

False

In [7]:
# Import Azure OpenAI
# from langchain_community.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

In [9]:
# Test the model connection
model = AzureChatOpenAI(
    openai_api_version=OPENAI_API_VERSION,
    deployment_name=OPENAI_DEPLOYMENT_NAME,
)
  

/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [10]:
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
model([message])



/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="J'aime la programmation.")

In [14]:
def get_police_forces():
    url = "https://data.police.uk/api/forces"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None

def search_police_forces(query):
    police_forces = get_police_forces()

    if not police_forces:
        print("Error: Unable to fetch police forces")
        return

    matching_forces = []

    for force in police_forces:
        if query.lower() in force["name"].lower():
            matching_forces.append(force)

    return matching_forces

class search_police_force(BaseTool):
    name = "search_police_force"
    description = "useful for when you need to answer questions about police forces, such as the id Unique force identifier and Police Force name"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return search_police_forces(query)
    
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
force_id = "devon"
police_force_data = search_police_forces(force_id)
print (police_force_data)

[{'id': 'devon-and-cornwall', 'name': 'Devon & Cornwall Police'}]


In [15]:
def get_police_force_id(force_id):
    url = f"https://data.police.uk/api/forces/{force_id}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None


class get_police_force_details(BaseTool):
    name = "get_police_force_details"
    description = "useful for when you need to answer questions about a specific police force such as the website, ways to keep informed, telephone number, Unique force identifier or Force name"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return get_police_force_id(query)
    
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
force_id = "devon-and-cornwall"
police_force_data = get_police_force_id(force_id)
print (police_force_data)

{'description': '<p>Devon &amp; Cornwall Police covers the largest geographical police area in England, extending 180 miles from the Dorset and Somerset borders in the east to the Isles of Scilly in the west.<br />\t\t<br />\t\tTo give some impression of the scale of the area, police headquarters in Exeter is actually nearer to London than the furthest extremity of the Force.<br />\t\t<br />\t\tPolicing an area of this size offers many challenges, with the needs of both rural and urban communities continually at the forefront of the Force\'s approach to tackling crime.</p>\n\n<p>\t\tThe Force aim is:<br />\t\t"By working in partnership to: bring about safer communities; reduce disorder, crime and the fear of crime; and contribute to the delivery of justice in a way which secures and maintains public confidence."</p>', 'url': 'http://www.devon-cornwall.police.uk', 'engagement_methods': [{'url': 'http://www.facebook.com/pages/Exeter-United-Kingdom/Devon-Cornwall-Police/151265442987', 'ty

In [16]:
def get_police_force_people(force_id):
    url = f"https://data.police.uk/api/forces/{force_id}/people"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None


class get_police_force_peoples(BaseTool):
    name = "get_police_force_peoples"
    description = "useful for when you need to answer questions about Senior officer bio"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return get_police_force_people(query)
    
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
force_id = "leicestershire"
police_force_data = get_police_force_people(force_id)
print (police_force_data)

[{'bio': '<p>Rob has served with Leicestershire Police for 25 years, beginning his policing career here in 1993.</p>\n\n<p>In 2008 he was appointed as the head of the force’s Community Safety Bureau, which was responsible for the implementation of neighbourhood policing, tackling anti-social behaviour and offender management.</p>\n\n<p>His next role, from 2009 to 2014, was as Chief Superintendent for the Leicester City Basic Command Unit. He led local policing for the area and worked with communities during two large scale English Defence League and Unite Against Fascism demonstrations. He was instrumental in the force’s response to disorder during August 2012.</p>\n\n<p>Throughout 2014 and 2015 he was head of business change and developed Blueprint2020 – a programme designed to decide how to make significant savings and take the force into a new era of policing.</p>\n\n<p>His next step was to become Temporary Assistant Chief Constable, working across Leicestershire, Northamptonshire a

In [17]:
def get_neighbourhoods(force_id):
    url = f"https://data.police.uk/api/{force_id}/neighbourhoods"
    cleaned_url = url.replace("'", "")
    response = requests.get(cleaned_url)
    neighbourhoods = response.json()

    if response.status_code == 200:
        names = [entry['name'] for entry in neighbourhoods]
        names_string = ', '.join(names)
        return (names_string)
    else:
        return None

class get_police_neighbourhoods(BaseTool):
    name = "get_police_neighbourhoods"
    description = "useful for when you need to answer questions about the name for the police force neighbourhood. You will need to use GetPoliceforce first to get the force name for the API"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return get_neighbourhoods(query)
    
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
force_id = "\'devon-and-cornwall\'"
neighbourhood_data = get_neighbourhoods(force_id)
print (neighbourhood_data)

Camborne North, Camborne South, Camborne West, Illogan North, Illogan South, Redruth North, Redruth South &amp; St Day &amp; Lanner, Helston Town, Lizard, Porthleven &amp; Helston North, Penzance Rural East, Penzance Rural West, Penzance Town East, Penzance Town West, St Just, Pendeen &amp; Sennen, Carbis Bay, Copperhouse &amp; Gwinear, Hayle Foundry, Isles of Scilly, St Ives Town, Arwenack &amp; Boslowick, Penwerris &amp; Trescobeas, Newquay Town, St Columb Major, Grampound &amp; Mevagissey, Lostwithiel &amp; Fowey, St Austell Town, St Blazey &amp; Tywardreath, St Stephen &amp; St Dennis, Treverbyn &amp; Roche, Boscawen, Moresk &amp; Tregolls, Mount Hawke &amp; St Agnes, Perranporth, Roseland &amp; Probus &amp; Trispen &amp; St Erme, Treliske Hospital, Bodmin Rural, Bodmin St Marys, Bodmin St Petroc, Padstow &amp; St Merryn, Wadebridge Town, Rock, Polzeath &amp; Port Isaac, Bude Rural, Bude Town, Camelford &amp; Tintagel, Launceston, Launceston Rural, Liskeard North, Liskeard Rural, L

In [18]:
def locate_neighbourhood(latlong):

    url = f"https://data.police.uk/api/locate-neighbourhood?q={latlong}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None

class locate_police_neighbourhood(BaseTool):
    name = "locate_police_neighbourhood"
    description = "useful for when you need to answer questions about the location of a police neighbourhood using lattitude, longitude numbers. Pass lattitude and longitude as a string"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return locate_neighbourhood(query)
    
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
latlong = "50.3656 ,-4.1423"
neighbourhood_data = locate_neighbourhood(latlong)
print (neighbourhood_data)

{'force': 'devon-and-cornwall', 'neighbourhood': 'PLY.4028'}


In [19]:
# Load the tool configs that are needed.
tools = [
    Tool(
        name="SearchPoliceForce",
        func=search_police_force.run,
        description="useful for when you need to answer questions about police forces, such as the id Unique force identifier and Police Force name"
    ),
    Tool(
        name="GetPoliceforceDetails",
        func=get_police_force_details.run,
        description="useful for when you need to answer questions about a specific police force such as the website, ways to keep informed, telephone number, Unique force identifier or Force name"
    ),
    Tool(
        name="GetPoliceforcePeople",
        func=get_police_force_peoples.run,
        description="useful for when you need to answer questions about Senior officer bio"
    ),
        Tool(
        name="GetPoliceforceNeighbourhood",
        func=get_police_neighbourhoods.run,
        description="useful for when you need to answer questions about the name for the police force neighbourhood. You will need to use GetPoliceforceDetails first to get the force name for the API"
    ),
    Tool(
        name="LocatePoliceNeighbourhood",
        func=locate_police_neighbourhood.run,
        description="useful for when you need to answer questions about the location of a police neighbourhood using lattitude, longitude numbers. Pass lattitude and longitude as a string"
    )
]

In [21]:
tools = [search_police_force(), get_police_force_details(), get_police_neighbourhoods(), locate_police_neighbourhood()]
agent = initialize_agent(tools, model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [23]:
agent.run("Get the police force details for latitude 50.3656, longitude -4.1423")



> Entering new AgentExecutor chain...
First, I need to use the latitude and longitude to find out which police neighbourhood the location falls under. Then I can identify the police force associated with that neighbourhood.

Action: locate_police_neighbourhood
Action Input: "50.3656, -4.1423"
Observation: {'force': 'devon-and-cornwall', 'neighbourhood': 'PLY.4028'}
Thought:Now that I have the police force identifier, I can get the details of the Devon and Cornwall Police force.

Action: get_police_force_details
Action Input: devon-and-cornwall
Observation: {'description': '<p>Devon &amp; Cornwall Police covers the largest geographical police area in England, extending 180 miles from the Dorset and Somerset borders in the east to the Isles of Scilly in the west.<br />\t\t<br />\t\tTo give some impression of the scale of the area, police headquarters in Exeter is actually nearer to London than the furthest extremity of the Force.<br />\t\t<br />\t\tPolicing an area of this size offers 

'The police force details for the location with latitude 50.3656 and longitude -4.1423, which falls under the Devon and Cornwall Police jurisdiction, are as follows:\n- Description: Devon & Cornwall Police covers the largest geographical police area in England, extending 180 miles from the Dorset and Somerset borders in the east to the Isles of Scilly in the west. The headquarters in Exeter is nearer to London than the furthest extremity of the Force. The Force aims to work in partnership to bring about safer communities, reduce disorder, crime and the fear of crime, and contribute to the delivery of justice in a way which secures and maintains public confidence.\n- Website URL: http://www.devon-cornwall.police.uk\n- Engagement Methods: \n  - Facebook: http://www.facebook.com/pages/Exeter-United-Kingdom/Devon-Cornwall-Police/151265442987\n  - Telephone: (Non-emergency contact number) - 101\n  - Twitter: http://twitter.com/DC_Police\n  - YouTube: http://www.youtube.com/dcpolice\n- Telep